In [5]:
from algorithm.louvain import louvain
from algorithm.spectral_clustering import spectral_clustering, normalized_spectral_clustering
from algorithm.hierarchical_clustering import mst_cut_clustering
from utils.analyze import get_rs_from_fixed_weighted_degree, compute_clustering, validation, metrics_to_df 
from utils.generation import multivariate_t_rvs, corr_estimate_parallel, get_mean_cov, get_cor_from_cov, corr_estimate_parallel, generate_samples_bag, get_corr_estimate, set_zero_weights_to_very_low
from utils import parallel

from scipy import stats
import pandas as pd
import numpy as np

In [7]:
rs = get_rs_from_fixed_weighted_degree(degree=16, cluster_size= 20, num_clusters=2,r_out_bound = (0,1,200))

In [8]:
mean_covs = [get_mean_cov(num_clusters = 2, cluster_size = 20, r_in = rs[0][i], r_out = rs[1][i]) for i in range(rs.shape[1])]
means = [mean_cov[0] for mean_cov in mean_covs]
covs = [mean_cov[1] for mean_cov in mean_covs]
true_graphs = [get_cor_from_cov(cov) for cov in covs]
samples_bags = [generate_samples_bag(means[i], covs[i], bags = 100, sample_size=40, distribution = np.random.multivariate_normal) for i,cov in enumerate(covs)]

In [9]:
estimator = corr_estimate_parallel(samples_bags, stats.pearsonr, backend='threading')
estimated_graphs_bags = estimator.get_estimations()

In [11]:
len(estimated_graphs_bags), len(estimated_graphs_bags[0]), estimated_graphs_bags[0][0].shape

(82, 100, (40, 40))

In [12]:
estimated_graphs_bags2 = [[set_zero_weights_to_very_low(get_corr_estimate(sample, stats.pearsonr)) for sample in samples_bag] for samples_bag in samples_bags]

In [13]:
len(estimated_graphs_bags2), len(estimated_graphs_bags2[0]), estimated_graphs_bags2[0][0].shape

(82, 100, (40, 40))

In [4]:
estimator = corr_estimate_parallel(samples_bags, stats.pearsonr, backend='threading')
estimated_graphs_bags = estimator.get_estimations()